Imports

In [1]:
import sys
import os
import time
import re
import json
from urllib.parse import quote as urlencode
from urllib.request import urlretrieve

import http.client as httplib 

from astropy.table import Table,join
import numpy as np

import pprint
pp = pprint.PrettyPrinter(indent=4)
import SciServer
from SciServer import CasJobs, SkyQuery, SciDrive, SkyServer
import requests
import pandas as pd
import astropy as ap
import astropy.io.fits as astrofits
import astropy.io.votable as astrovot
import astropy.wcs as astrowcs
import astropy.units as astrounits
import astropy.coordinates as astrocoords
import astropy.visualization as astrovis
import astropy.visualization.mpl_normalize as astromplnorm
import astropy.nddata as astronddata
import astropy.nddata.utils as astrondutils
import io
import pdb 
from IPython.core.display import display
from astropy.table import Table,vstack,Column
from astroquery.mast import Catalogs
from astropy.utils.console import ProgressBar
from collections import deque
from pydoc import locate

import matplotlib.pyplot as mplplot

import gzip
import tarfile

Definitions

In [3]:
def mastQuery(request):
    """Perform a MAST query.
    
        Parameters
        ----------
        request (dictionary): The MAST request json object
        
        Returns head,content where head is the response HTTP headers, and content is the returned data"""
    
    server='mast.stsci.edu'

    # Grab Python Version 
    version = ".".join(map(str, sys.version_info[:3]))

    # Create Http Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}

    # Encoding the request as a json string
    requestString = json.dumps(request)
    requestString = urlencode(requestString)
    
    # opening the https connection
    conn = httplib.HTTPSConnection(server)

    # Making the query
    conn.request("POST", "/api/v0/invoke", "request="+requestString, headers)

    # Getting the response
    resp = conn.getresponse()
    head = resp.getheaders()
    content = resp.read().decode('utf-8')

    # Close the https connection
    conn.close()

    return head,content

def HscCrossmatch(data):
    
    crossmatchInput = {"fields":[{"name":"ra","type":"float"},
                                 {"name":"dec","type":"float"}],
                       "data":data}
    
    request = {"service":"Mast.Hsc.Crossmatch",
               "data":crossmatchInput,
               'params':{
                   "raColumn":"ra",
                   "decColumn":"dec",
                   'radius':0.001,
                   'cache-breaker':10},
               "pagesize":1000,
               "page":1,
               "format":"json",
               "removecache":True}
    
    headers,outString = mastQuery(request)
    outData = json.loads(outString)
    
    return outData

# def mastJson2Table_Filt(jsonObj):
#     filt = 'A_F475W'
#     Obj = [x for x in jsonObj['data'] if x.get(filt,None) != None]
#     mastDataTable = Table()

#     for col,atype in [(x['name'],x['type']) for x in jsonObj['fields']]:
#         mastDataTable[col] = np.array([x.get(col,None) for x in Obj])
    
#     return mastDataTable


def getHSCMatches(matchId):

#     data = result['data']

#     # get the match id
#     matchId = data[0]['MatchID']
#     print(type(matchId))
    # get detailed results for chosen match id
    request = {'service':'Mast.HscMatches.Db',
               'params':{'input':matchId,
                         'cache-breaker':10},
               'format':'json',
               'page':1,
               'pagesize':4}   

    headers,outString = mastQuery(request)

    outData = json.loads(outString)

    return outData

def genBundleRequest(imageNames, outfileNamePrefix='downloadBundle', extension='tar.gz'):
    baseUrl = "http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset="
    urlList = ",".join(['{}{}'.format(baseUrl, imageName) for imageName in imageNames])
    pathList = ['{}.fits'.format(imageName) for imageName in imageNames]

    request = {"service":"Mast.Bundle.Request",
               "params":{"urlList":urlList,
                         "filename":outfileNamePrefix,
                         "pathList":pathList,
                         "extension":extension,
                         'cache-breaker':10}
              }
    return request,outfileNamePrefix,extension

def downloadRequest(url):
    server='mast.stsci.edu'
    
    conn = httplib.HTTPSConnection(server)
    conn.request("GET", url)
    resp = conn.getresponse()
   
    fileName = resp.getheader('Content-Disposition')[21:]
    fileContent = resp.read()

    with open(fileName,'wb') as FLE:
        FLE.write(fileContent)

    conn.close()

    return fileName

def query_cas():
    query = "SELECT dr8objid, ra, dec FROM zoo2MainSpecz"
    result = CasJobs.executeQuery(query, 'DR12', format='pandas')

    result = Table.from_pandas(result)
    result.write('casquery.ecsv')
    return result

def read_cas(filename='casquery.ecsv'):
    return Table.read(filename)

def query_mast(coords,radius=0.001*astrounits.deg):
    results = Catalogs.query_region(coords,radius=radius)
    print(results)
    
def stringtype(typestr):
    typemap = {'string':str,'float':float,'int':int,'boolean':bool}
    return typemap[typestr]
    
def typecast(val,typestr):
    if val is None:
        return None
    elif typestr == 'string':
        return str(val)
    else:
        return locate(typestr)(val)
    
def mastJson2Table(line):
    jsonObj = json.loads(line)
    if not jsonObj['data']:
        return None
    coldict = {field['name']:stringtype(field['type']) for field in jsonObj['fields']}
    keys,dtypes = zip(*coldict.items())
    
    rows = deque()
    
    for d in jsonObj['data']:
        row = [d[key] if d[key] is not None else '99' for key in keys]
        rows.append(row)
     
    table = Table(rows=list(rows),names=keys,dtype=dtypes)
    return table

def read_matches(lines):
    tables = deque()
    for line in ProgressBar(lines):
        table = mastJson2Table(line)
        if table:
            tables.append(table)
    table = vstack(list(tables))
    #coords = astrocoords(ra=table['ra'],dec=table['dec'],unit=(astrounits.deg,astrounits.deg))
    #table.add_column(coords,index=0,name='coord')
    return table

Variables

In [4]:
catalog = 'zoo2MainSpecz'
select = 'dr8objid'

Catalogs

In [ ]:
query = "SELECT " + select + ", ra, dec FROM " + catalog
result = CasJobs.executeQuery(query, 'DR12', format='pandas')
ra_SDSS = result.ra.values
dec_SDSS = result.dec.values
obj = result.dr8objid.values
display(result)

os.chdir('/home/idies/workspace/persistent/JSON')
bins = 500
size = int(len(ra)/500)
lines = []
for i in range(size):
    ra_dec_list = [{"ra":ras,"dec":decs} for ras,decs in zip(ra_SDSS,dec_SDSS)]
    data = ra_dec_list[i*bins:(i*bins)+bins]
    info = HscCrossmatch(data)
    lines.append(info)
stacked_table = read_matches(lines)

coords_HSC = SkyCoord(ra=stacked_table['ra'],dec=stacked_table['dec'],unit=(astrountis.deg,astrounits.deg))
coords_SDSS = SkyCoord(ra_SDSS,dec_SDSS,unit=(astrounits.deg,astrounits.deg))

idx, d2d, d3d = coords_HSC.match_to_catalog_sky(coords_ZOO) 
#Add things to table
objectid = Column(name=select, data=[obj[x] for x in idx])
stacked_table.add_column(objectid)



Download and Delete